In [16]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras

filename = './_data/filtered_data.csv'
data = pd.read_csv('./_data/filtered_data.csv')

In [23]:
data = data.drop(['Unnamed: 0'],axis=1)
data.head()

,Date,opptAbbr,teamAbbr,Pts,Spread,Total,Open_Line_OU,O/U,teamDayOff,teamPTS,...,opptFIC40,opptOrtg,opptDrtg,opptEDiff,opptPlay%,opptAR,opptAST/TO,opptSTL/TO,opptPoss,opptPace
0,2017-10-17,CLE,BOS,99.0,-3.0,201.0,216.5,0.0,0.0,99.0,...,55.52,102.7,99.7,3.0,0.42,17.59,1.12,0.18,99.3,99.3
1,2017-10-17,GSW,HOU,122.0,1.0,243.0,235.0,1.0,0.0,122.0,...,86.56,118.6,119.6,-1.0,0.47,27.92,2.00,0.29,102.0,102.0
2,2017-10-18,ORL,MIA,109.0,-7.0,225.0,207.0,1.0,0.0,109.0,...,78.23,110.3,103.6,6.7,0.46,19.43,1.57,0.57,105.2,105.2
3,2017-10-18,WAS,PHI,115.0,-5.0,235.0,215.0,1.0,0.0,115.0,...,80.00,115.1,110.3,4.8,0.47,19.04,2.33,0.89,104.3,104.3
4,2017-10-18,BOS,MIL,108.0,8.0,208.0,209.5,0.0,0.0,108.0,...,58.65,102.2,110.4,-8.2,0.42,19.70,1.92,1.00,97.9,97.9


In [120]:
from sklearn.model_selection import train_test_split

def model_preprocessing(dataset,date_range,test_size):
    """
    Takes filtered dataset and returns train_test_split dataset for training model. Removes columns of dtype == "object".
    
    :dataset: input dataset (pd.DataFrame type)
    :seasons: range of seasons eg. ("2016-01-01","2019-01-01")
    :test_size: validation set size
    """
    
    seasonal_data = dataset.loc[np.where((dataset['Date'] > date_range[0]) & (dataset['Date'] < date_range[1]))]
    
    column_ind = [i for i in range(0,len(dataset.dtypes)) if dataset.dtypes[i] == 'object']
    seasonal_data = seasonal_data.drop(columns = list(seasonal_data.columns[column_ind]), axis = 1)
    
    OU_classification = np.asarray(seasonal_data['O/U'])
    
    seasonal_data = seasonal_data.drop(columns = ['O/U'], axis = 1)
    seasonal_data = np.asarray(seasonal_data, dtype = float)   
    seasonal_data = keras.utils.normalize(seasonal_data, axis=1)
     
    return train_test_split(seasonal_data, OU_classification, random_state=42,test_size = test_size)

X_train, X_test, y_train, y_test = model_preprocessing(dataset = data,
                                                       date_range = ("2015-01-01", "2029-01-01"),
                                                       test_size = 0.1)
    

In [122]:
model = tf.keras.models.Sequential()

model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(128, activation=tf.nn.relu6))
model.add(tf.keras.layers.Dense(3, activation=tf.nn.softmax))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [123]:
model.fit(X_train, y_train, epochs=50, validation_split=0.1, batch_size=32)

Epoch 1/50
130/130 [==============================] - 2s 9ms/step - loss: 0.7947 - accuracy: 0.5348 - val_loss: 0.7345 - val_accuracy: 0.4610
Epoch 2/50
130/130 [==============================] - 1s 6ms/step - loss: 0.7259 - accuracy: 0.6227 - val_loss: 0.6925 - val_accuracy: 0.8009
Epoch 3/50
130/130 [==============================] - 1s 6ms/step - loss: 0.6896 - accuracy: 0.7434 - val_loss: 0.6667 - val_accuracy: 0.6277
Epoch 4/50
130/130 [==============================] - 1s 6ms/step - loss: 0.6416 - accuracy: 0.7981 - val_loss: 0.5995 - val_accuracy: 0.8095
Epoch 5/50
130/130 [==============================] - 1s 6ms/step - loss: 0.5889 - accuracy: 0.8258 - val_loss: 0.5362 - val_accuracy: 0.8550
Epoch 6/50
130/130 [==============================] - 1s 6ms/step - loss: 0.5437 - accuracy: 0.8114 - val_loss: 0.4974 - val_accuracy: 0.8247
Epoch 7/50
130/130 [==============================] - 1s 7ms/step - loss: 0.5033 - accuracy: 0.8270 - val_loss: 0.4594 - val_accuracy: 0.8485
Epoch 